In [12]:
from ast import arg
import sys, os
sys.path.append(os.getcwd())
import numpy as np 
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from utils.dist_utils import init_distributed_mode

from model import ACTIONCHORE
from trainer.trainer import Trainer
from data.data_paths import DataPaths
from data.train_data import BehaveDataset

from config.config_loader import load_configs


configs = load_configs('chore-release')
# assert args.exp_name==configs.exp_name
# add command line configs
configs.device = 'cuda'
configs.world_size = 4
configs.dist_url = 'env://'
configs.rank = 0

def index(feat, uv):
    '''
    :param feat: [B, C, H, W] image features
    :param uv: [B, 2, N] uv coordinates in the image plane, range [-1, 1]
    :return: [B, C, N] image features at the uv coordinates
    '''
    uv = uv.transpose(1, 2)  # [B, N, 2]
    uv = uv.unsqueeze(2)  # [B, N, 1, 2]
    samples = torch.nn.functional.grid_sample(feat, uv, align_corners=True)  # [B, C, N, 1] or [B, C, N, 5]

    return samples[:, :, :, 0]  # [B, C, N]

def test_model(args):
    world_size = torch.cuda.device_count()
    init_distributed_mode(args)
    rank = args.rank
    device = torch.device(args.device)
    model = ACTIONCHORE(args, rank=rank)

    model.check_action_chore()

def test_dataset(args):
    train_paths, val_paths = DataPaths.load_splits(args.split_file)
    
    return train_paths, val_paths

configs loaded from config/chore-release.json


In [13]:
train_paths, val_paths = test_dataset(configs)

In [17]:
path_dir = train_paths[0].split('/')[:-1]
path_dir = '/'.join(path_dir)
action = np.load(os.path.join(path_dir, 'action.npz'))

In [18]:
action['action']

array(2)

In [ ]:
test_model(configs)